In [ ]:
!pip install transformers wandb datasets evaluate jiwer accelerate==0.20.3 sentencepiece

In [ ]:
import pandas as pd
import json
import os
import re
import numpy as np
#import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel, Seq2SeqTrainer, Seq2SeqTrainingArguments, T5ForConditionalGeneration, GPT2Tokenizer, T5TokenizerFast, T5Tokenizer
import torch.nn.functional as F
import torch.nn as nn
import torch
from datasets import Dataset
import random
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter
from copy import deepcopy
from sklearn.metrics import precision_recall_curve, auc, roc_auc_score, f1_score, accuracy_score
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR
from transformers.models.bert.modeling_bert import BertPreTrainedModel
from transformers.models.deberta_v2.modeling_deberta_v2 import DebertaV2PreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from sklearn.metrics.pairwise import cosine_similarity
import wandb
from evaluate import load
from torch.utils.data import Dataset as BaseDataset
from transformers import Adafactor
import accelerate
cer = load('cer')

In [ ]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")
set_seed()

Random seed set as 42


Обучение

In [ ]:
class CFG:
    model_name = 'cointegrated/rut5-base-multitask'
    batch_size = 2
    sep = ';'
    domain = 'clear'

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataFORMAT.csv')

In [ ]:
df = df.drop(["Unnamed: 0.1", "Unnamed: 0"], axis=1)

In [ ]:
train_df, val_df = train_test_split(df, random_state=42, test_size=0.1)
train_df = train_df.drop_duplicates(['raw'])

In [ ]:
train_df

,raw,clear
58149,"<p>Квиз- это увлекательная викторина, позволяю...","Квиз- это увлекательная викторина, позволяюща..."
35089,<p>В рамках данного мероприятия волонтеры оказ...,В рамках данного мероприятия волонтеры оказыв...
45252,<p>Мероприятие направлено на профилактику опас...,Мероприятие направлено на профилактику опасно...
3700,"Интеллектуальная игра ""Новаторы"" для учащихся ...","Интеллектуальная игра ""Новаторы"" для учащихся ..."
7217,<p>Волонтеры Курского государственного медицин...,Волонтеры Курского государственного медицинск...
...,...,...
44131,<p>В летнее время множество людей выбрасывают ...,В летнее время множество людей выбрасывают му...
6265,В связи с нехваткой образовательных видеоролик...,В связи с нехваткой образовательных видеоролик...
54886,<p>15 мая в ФОС «Ледовый дворец» Тулы пройдет ...,15 мая в ФОС «Ледовый дворец» Тулы пройдет фи...
860,<p>Необходима помощь волонтеров на пунктах вак...,Необходима помощь волонтеров на пунктах вакци...


In [ ]:
val_df

,raw,clear
47182,"<p>Мастер-класс ""Живая нить традиций"" основан ...","Мастер-класс ""Живая нить традиций"" основан на..."
985,Мы свяжемся с вами для уточнения адреса,Мы свяжемся с вами для уточнения адреса
771,<p><span>Акция памяти «Блокадный хлеб» актуали...,Акция памяти «Блокадный хлеб» актуализирует ...
68072,м. Рыбацкое,м. Рыбацкое
9670,"<p>Акция ""У войны не женское лицо"" проводится ...","Акция ""У войны не женское лицо"" проводится с ..."
...,...,...
59737,<p>Волонтерская деятельность во время летних...,Волонтерская деятельность во время летних к...
28374,<p>Программа:</p><p>11.00 - <span>Парад Колясо...,Программа: 11.00 - Парад Колясок 12.00...
42921,<p>Новоуренгойцы от 18 лет и разного уровня по...,Новоуренгойцы от 18 лет и разного уровня подг...
27783,"<p>Конкурс ""Костюмчик с иголочки"" Ребята прихо...","Конкурс ""Костюмчик с иголочки"" Ребята приходя..."


In [ ]:
dataset_train = Dataset.from_pandas(train_df)
dataset_val = Dataset.from_pandas(val_df)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(CFG.model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
 #tokenized_examples = tokenizer(test.iloc[5, 2])
print(len(tokenized_examples['input_ids']))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 tokenized_examples = tokenizer(test.iloc[5, 2])                                              │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'test' is not defined

In [ ]:
def prepare_features(examples):
    tokenized_examples = tokenizer(
        ['<LM>' +x for x in examples["raw"]],
        padding='max_length',
        max_length=128,
        truncation=True,
        return_tensors='np'
    )
    target_encoding = tokenizer(
        [f'{i}</s>' for i in examples[CFG.domain]],
        padding="max_length",
        max_length=128,
        truncation=True,
        return_tensors='np'
    )
    labels = target_encoding.input_ids
    labels[labels == tokenizer.pad_token_id] = -100
    tokenized_examples['labels'] = labels
    return tokenized_examples

In [ ]:
tokenized_dataset_train = dataset_train.map(
    prepare_features, batched=True, remove_columns=dataset_train.column_names)
tokenized_dataset_val = dataset_val.map(
    prepare_features, batched=True, remove_columns=dataset_val.column_names)

Map:   0%|          | 0/53849 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:283: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Map:   0%|          | 0/7137 [00:00<?, ? examples/s]

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(CFG.model_name)

In [ ]:
args = Seq2SeqTrainingArguments(
    'outputs/brand_10',
    evaluation_strategy = 'epoch',
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    learning_rate =1e-4,
    weight_decay=0.01,
    num_train_epochs=10,
    logging_steps = 2,
    save_strategy = 'epoch',
    report_to = 'wandb',
    gradient_accumulation_steps=4,
    predict_with_generate = True,
    save_total_limit = 1,
    optim='adafactor',
)

In [ ]:
class F1Score:
    def __init__(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0

    def update(self, pred, target):
        pred = frozenset(x for x in pred)
        target = frozenset(x for x in target)
        self.tp += len(pred & target)
        self.fp += len(pred - target)
        self.fn += len(target - pred)

    def reset(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0

    def get(self):
        if self.tp == 0:
            return 0.0
        precision = self.tp / (self.tp + self.fp)
        recall = self.tp / (self.tp + self.fn)
        return 2 / (1 / precision + 1 / recall)

In [ ]:
def compute_metrics(preds):
    labels, predictions = preds.label_ids, preds.predictions
    labels[labels == -100] = tokenizer.pad_token_id
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    print(labels[-1], predictions[-1])
    y_true = [x.strip() for x in labels]
    y_pred = [x.strip() for x in predictions]
    f1 = F1Score()
    f1.update(y_pred, y_true)
    return {f'cer_{CFG.domain}': cer.compute(predictions=y_pred, references=y_true), f'f1_{CFG.domain}': f1.get(), f'accuracy_{CFG.domain}': accuracy_score(y_true, y_pred)}

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset = tokenized_dataset_train,
    eval_dataset=tokenized_dataset_val,
    compute_metrics = compute_metrics,
)

In [ ]:
trainer.train()

Инференс

In [ ]:
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-base-multitask")
tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-base-multitask")

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/pytorch_model.bin', map_location=torch.device('cuda')))
model.eval()
model.to("cuda")

In [ ]:
def generate(text, **kwargs):
    inputs = tokenizer(text,max_length=512, return_tensors='pt')
    with torch.no_grad():
        hypotheses = model.generate(**inputs.to("cpu"),max_length=512,do_sample=True,num_beams=5,**kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

In [ ]:
# Декодируем данные
tokenizer.batch_decode(predicts, skip_special_tokens=True)

['Сроки привлечения волонтеров: с 29 августа по 9 сентября 2022Э']